In [26]:
import numpy as np
import pickle
import sys

In [27]:
x_th_max = -2.5
x_th_min = -8.5 
var_th = 1
min_bite_interval = 40
left_length = 48
right_length = 48
max_annot_distance = 32
exclude_annot_distance = 64

In [28]:
def get_data(type):
    path = "C:\\ASM\\DevData\\eating\\python\\"+type+"_data.pkl"
    with open(path, "rb") as file:
        data = pickle.load(file)        
    return data    

In [29]:
def find_min_points(x):
    step_length = min_bite_interval//2
    count = len(x)
    
    mp = []
    for i in range(0, count-step_length, step_length):
        min_index = i
        for j in range(i, i+step_length):
            if x[j] < x[min_index]:
                min_index = j
                
        if x[min_index] <= x_th_max and min_index>=80 and min_index<=count-80:
            mp.append(min_index)
    
    print("Initial mp count: ", len(mp))   
    if len(mp)<=1:
        return mp
    
    while True:
        res = []
        count = len(mp)
        ix = mp[0]
        ixRight = mp[1]
        if ixRight - ix > min_bite_interval or x[ix] < x[ixRight]:
            res.append(ix)
        
        for i in range(1, count - 1):
            ix = mp[i]
            ixLeft = mp[i - 1]
            ixRight = mp[i + 1]

            cond_left = ix - ixLeft > min_bite_interval or x[ix] <= x[ixLeft]
            cond_right = ixRight - ix > min_bite_interval or x[ix] < x[ixRight]        

            if cond_left and cond_right:
                res.append(ix)
        
        ix = mp[count - 1]
        ixLeft = mp[count - 2]
        if ix - ixLeft > min_bite_interval or x[ix] <= x[ixLeft]:
            res.append(ix)            
        
        if len(mp) == len(res):
            break        
        mp = res
        print("     mp count: ", len(mp))
        
    print("MP count before filter: ", len(res))    
    return res

In [30]:
def filter_min_points(mp, d):
    res = [];
    count = len(mp)
    for i in range(count):
        ix = mp[i]
        v = np.sum(np.var(d[ix-left_length:ix+right_length+1, 1:], axis = 0))
        if v>=var_th:
            res.append(ix)
       
    print("MP count after filter: ", len(res))    
    return res
    

In [31]:
def label_min_points_by_bite(mp, annots):
    mp_count = len(mp)
    if mp_count == 0:
        return mp
    
    res = np.zeros((mp_count, 3))      
    annot_count = len(annots)
    print("annot count: ", annot_count, ", mp count: ", mp_count)
    #print(annots[:10, :])
    
    for i in range(mp_count):
        res[i, 0] = mp[i]
        j = np.argmin( np.abs(annots[:, 0] - mp[i]) )
        if np.abs(annots[j, 0] - mp[i]) <= max_annot_distance:
            res[i, 1] = annots[j, 1]
            res[i, 2] = 1
        elif np.abs(annots[j, 0] - mp[i]) <= exclude_annot_distance:
             res[i, 1] = -1
    
    res = res[res[:, 1]>=0, :]
    while True:
        flag = True
        mp_count = len(res)        
        for i in range(1, mp_count-1):
                if res[i, 1] == 0 and ((res[i-1, 1]>0 and res[i, 0] - res[i-1, 0]<=80) or (res[i+1, 1]>0 and res[i+1, 0] - res[i, 0]<=80)):
                    res[i, 1] = -1
                    flag = False
        
        res = res[res[:, 1]>=0, :]        
        if flag:
            if res[0, 1] == 0 and res[1, 1]>0 and res[1, 0] - res[0, 0]<=80:
                res[0, 1] = -1
                
            mp_count = len(res)
            if res[mp_count-1, 1] == 0 and res[mp_count-2, 1]>0 and res[mp_count-1, 0] - res[mp_count-2, 0]<=80:
                res[mp_count-1, 1] = -1
                
            res = res[res[:, 1]>=0, :]            
            break
                    
                        

    print("Bite MP count: ", sum(res[:, 1]==1), "/", sum(annots[:, 1]==1) )
    print("Drink MP count: ", sum(res[:, 1]==2), "/", sum(annots[:, 1]==2) )
    print("Pos MP count: ", sum(res[:, 1]>=1), "/", sum(annots[:, 1]>=1) )        
    print("Neg MP count: ", sum(res[:, 1]==0))                
    print("Exclude MP count: ", sum(res[:, 1]==-1))                
    
    res = res.astype(int)
    return res
    

In [32]:
def label_min_points_by_meal(mp, annots):
    mp_count = len(mp)
    if mp_count == 0:
        return mp
    
    res = np.zeros((mp_count, 3))    
    annot_count = len(annots)    
    
    for i in range(mp_count):
        res[i, 0] = mp[i]            
        if  annot_count > 0:
            for j in range(annot_count):
                if annots[j, 0] <= mp[i] <= annots[j, 1]:
                    res[i, 1] = annots[j, 2]
                    res[i, 2] = 1
                    break

    print("Meal MP count: ", sum(res[:, 1]==1))
    print("Snack MP count: ", sum(res[:, 1]==2))                
    print("Drink MP count: ", sum(res[:, 1]==3))
    print("Pos MP count: ", sum(res[:, 1]>0))    
    print("Neg MP count: ", sum(res[:, 1]==0))
    res = res.astype(int)
    return res


In [33]:
def find_min_points_for_dataset(ds):
    subject_count = len(ds)
    ds_mp = []
    for subject in range(subject_count):
        subject_data = ds[subject]
        sess_count = len(subject_data)
        subject_mp =[]
        for sess in range(sess_count):
            print("\nSubject: {}, Sess: {}".format(subject, sess))
            sess_data = subject_data[sess]
            accel = sess_data[0]
            annots = sess_data[1]
            sess_mp = find_min_points(accel[:, 1])
            sess_mp = filter_min_points(sess_mp, accel)
            if len(annots)==0 or annots.shape[1] == 3:
                sess_mp = label_min_points_by_meal(sess_mp, annots)
            else:
                sess_mp = label_min_points_by_bite(sess_mp, annots)
                
            subject_mp.append(sess_mp)
        ds_mp.append(subject_mp)        
    return ds_mp    

In [34]:
mps = []
ds_names = ["uva_lab", "steven_lab", "uva_free", "steven_free"]

for i in range(4):
    ds = get_data(ds_names[i])
    print("Data set: ", ds_names[i])
    mp = find_min_points_for_dataset(ds)
    mps.append(mp)

with open("C:\\ASM\\DevData\\eating\\python\\selected_min_points.pkl", "wb") as file:
    pickle.dump(mps, file)



Data set:  uva_lab

Subject: 0, Sess: 0
Initial mp count:  1174
     mp count:  292
     mp count:  282
MP count before filter:  282
MP count after filter:  202
annot count:  102 , mp count:  202
Bite MP count:  55 / 75
Drink MP count:  13 / 27
Pos MP count:  68 / 102
Neg MP count:  101
Exclude MP count:  0

Subject: 1, Sess: 0
Initial mp count:  1152
     mp count:  310
     mp count:  289
MP count before filter:  289
MP count after filter:  246
annot count:  115 , mp count:  246
Bite MP count:  65 / 84
Drink MP count:  23 / 31
Pos MP count:  88 / 115
Neg MP count:  110
Exclude MP count:  0

Subject: 2, Sess: 0
Initial mp count:  780
     mp count:  209
     mp count:  198
MP count before filter:  198
MP count after filter:  157
annot count:  81 , mp count:  157
Bite MP count:  54 / 69
Drink MP count:  10 / 12
Pos MP count:  64 / 81
Neg MP count:  69
Exclude MP count:  0

Subject: 3, Sess: 0
Initial mp count:  493
     mp count:  155
     mp count:  150
MP count before filter:  150
MP

In [ ]:
print(mps[2][0][0])